# Symbolics in Julia


In [1]:
using Symbolics

In [2]:
@variables x y

2-element Vector{Num}:
 x
 y

In [3]:
A = [x^2 + y 0 2x
     0 0 2y
    y^2 + x 0 0]

3×3 Matrix{Num}:
 y + x^2  0  2x
       0  0  2y
 x + y^2  0   0

## Derivatives

In [4]:
@variables t

1-element Vector{Num}:
 t

In [5]:
D = Differential(t)

(::Differential) (generic function with 2 methods)

In [6]:
z = t + t^2

t + t^2

In [7]:
D(z)

Differential(t)(t + t^2)

The derivative is a lazy operator, as expected. We can expand it using `expand_derivatives`:

In [8]:
expand_derivatives(D(z))

1 + 2t

In [9]:
Symbolics.jacobian(A[:,1], [x, y])

3×2 Matrix{Num}:
 2x   1
  0   0
  1  2y

In [10]:
Symbolics.hessian(x^2 + 3y^2 - x, [x, y])

2×2 Matrix{Num}:
 2  0
 0  6

## Simplification and substitution

In [11]:
simplify(2x + 2y)

2x + 2y

In [12]:
B = simplify.([t + t^2 + t + t^2 2t + 4t
 x + y + y + 2t x^2 - x^2 + y^2])

2×2 Matrix{Num}:
 2t + 2(t^2)   6t
 x + 2t + 2y  y^2

In [15]:
V = substitute.(B, (Dict(x => 2.0, y => 3.0, t => 4.0),))

2×2 Matrix{Num}:
 40.0  24.0
 16.0   9.0

## Functions

In [17]:
f(x, y) = x^2 + y
@register_symbolic f(x, y)

In [19]:
f(x, y) + y^2

y^2 + f(x, y)

If we were to do derivatives, we need additional steps:

In [23]:
Symbolics.derivative(::typeof(f), args::NTuple{2, Any}, ::Val{1}) = 2args[1]

In [29]:
dx = Differential(x)

(::Differential) (generic function with 2 methods)

In [30]:
expand_derivatives(dx(f(x, y)))

2x

In [33]:
Symbolics.derivative(f(x, y), x)

2x

Notice how `expand_derivatives` of the $\text{d}x$ on $f(x,y)$ gives us $2x$ as the derivative we registered was defined as `2args[1]`. So we need to manually define what the derivative is, before we can make it _manageable_ with the differentiation system.

To make it consistent, then:

In [35]:
Symbolics.derivative(::typeof(f), args::NTuple{2, Any}, ::Val{2}) = 1

So now we can do:

In [36]:
Symbolics.derivative(f(x, y), y)

1

In [37]:
dy = Differential(y)

(::Differential) (generic function with 2 methods)

In [39]:
expand_derivatives(dy(f(x,y)), y)

1

## Notes

From [Symbolics.jl documentation](https://symbolics.juliasymbolics.org/stable/comparison/):
> While Symbolics.jl has many features missing from SymPy, it does not superset SymPy's functionality. For a list of missing features, see this issue.

This is important, because sometimes we want to mix the symbolic programming with the numerical methods. Sometimes, we don't. If we wanna keep it separated, we can go back to SymPy :)